<a href="https://colab.research.google.com/github/ancestor9/24_fall_textmining_NLP/blob/main/1022_%EB%8F%99%EA%B8%B0_%EB%B9%84%EB%8F%99%EA%B8%B0%EB%B0%A9%EC%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  
## **동기방식(Sync 방식)**
- 코드가 파일을 하나씩 순차적으로 읽으므로 파일 수가 많아지면 시간이 많이 소요

## **비동기(Asynchronous) 방식**
- 여러 작업을 동시에 실행, 특정 작업(예: 네트워크 요청)을 기다리는 동안 다른 작업을 진행할 수 있도록 하여 프로그램의 효율성을 높임
- Python에서는 asyncio와 같은 모듈을 사용하여 비동기 처리를 구현

##### <font color='red'> 커피샵에서 친구에게 커피를 부탁할 경우 친구랑 같이 카피 받으러가나? **(동기, sync)** 아니면 친구가 커피를 가져오는 동안 다른 작업을 하나? **(비동기, async)**

<img src ='https://images.unsplash.com/photo-1516367519848-ee5b4deb59d2?w=500&auto=format&fit=crop&q=60&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxzZWFyY2h8M3x8Y29mZWUlMjBzaG9wfGVufDB8fDB8fHww'>

In [ ]:
# 여러 개의 파일 URL 지정 (예시)
file_urls = [
    "https://people.sc.fsu.edu/~jburkardt/data/csv/airtravel.csv",
    "https://people.sc.fsu.edu/~jburkardt/data/csv/grades.csv",
    "https://people.sc.fsu.edu/~jburkardt/data/csv/hw_200.csv",
    "https://people.sc.fsu.edu/~jburkardt/data/csv/addresses.csv",
    "https://people.sc.fsu.edu/~jburkardt/data/csv/ford_escort.csv",
    "https://people.sc.fsu.edu/~jburkardt/data/csv/biostats.csv",
    "https://people.sc.fsu.edu/~jburkardt/data/csv/trees.csv",
    "https://people.sc.fsu.edu/~jburkardt/data/csv/weather.csv",
    "https://people.sc.fsu.edu/~jburkardt/data/csv/snakes_count_100.csv",
    "https://people.sc.fsu.edu/~jburkardt/data/csv/zillow.csv"
]

### **1. 동기 방식으로 네트워크에서 파일 다운 받기**

In [ ]:
import requests

def download_file_sync(url, save_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print(f"{save_path} 다운로드 완료")

def download_all_files_sync():
    for url in file_urls:
        filename = os.path.basename(url)
        save_path = f"./{filename}"
        download_file_sync(url, save_path)

# 동기 방식으로 파일 다운로드 시작
t1 = time.time()
download_all_files_sync()
t2 = time.time()
print(f"동기 방식으로 파일 다운로드 완료: {t2 - t1:.2f}초 소요")

./airtravel.csv 다운로드 완료
./grades.csv 다운로드 완료
./hw_200.csv 다운로드 완료
./addresses.csv 다운로드 완료
./ford_escort.csv 다운로드 완료
./biostats.csv 다운로드 완료
./trees.csv 다운로드 완료
./snakes_count_100.csv 다운로드 완료
./zillow.csv 다운로드 완료
동기 방식으로 파일 다운로드 완료: 5.96초 소요


### **2. 비동기 방식으로 네트워크에서 파일 다운 받기**

In [ ]:
import aiohttp
import asyncio
import time
import os

async def download_file(session, url, save_path):
    async with session.get(url) as response:
        if response.status == 200:
            content = await response.read()
            with open(save_path, 'wb') as f:
                f.write(content)
            print(f"{save_path} 다운로드 완료")

async def download_all_files():
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in file_urls:
            filename = os.path.basename(url)
            save_path = f"./{filename}"
            tasks.append(download_file(session, url, save_path))
        await asyncio.gather(*tasks)


# 비동기 방식으로 파일 다운로드 시작
# asyncio.run()을 await asyncio.gather()로 변경하여 Jupyter Notebook 환경에서 실행
t1 = time.time()
# await download_all_files()  # Jupyter Notebook에서는 이렇게 직접 호출하면 안됩니다.
# Instead of using asyncio.run, use await within a coroutine or the IPython event loop
await download_all_files()  # Jupyter/IPython 환경에서 실행하기 위해 await 사용
t2 = time.time()
print(f"비동기 방식으로 파일 다운로드 완료: {t2 - t1:.2f}초 소요")



./ford_escort.csv 다운로드 완료
./grades.csv 다운로드 완료
./zillow.csv 다운로드 완료
./hw_200.csv 다운로드 완료
./trees.csv 다운로드 완료
./airtravel.csv 다운로드 완료
./biostats.csv 다운로드 완료
./snakes_count_100.csv 다운로드 완료
./addresses.csv 다운로드 완료
비동기 방식으로 파일 다운로드 완료: 0.94초 소요


## **3. 다운로딩 파일의 순서를 동기방식과 비동기 방식을 비교하라**

In [ ]:
import aiohttp
import asyncio
import time
import os
import requests
from bs4 import BeautifulSoup


# 다운로드할 모든 파일 URL을 가져오기 위해 HTML 파싱
def get_all_csv_urls(base_url):
    # base_url의 HTML을 요청하여 페이지의 내용을 가져옵니다.
    response = requests.get(base_url)
    if response.status_code == 200:
        # BeautifulSoup을 사용하여 HTML을 파싱합니다.
        soup = BeautifulSoup(response.text, 'html.parser')
        file_urls = []
        # 모든 'a' 태그를 찾아서 href 속성을 확인합니다.
        for link in soup.find_all('a'):
            href = link.get('href')
            # 링크가 '.csv'로 끝난다면 CSV 파일로 간주하고 URL 목록에 추가합니다.
            if href.endswith('.csv'):
                file_urls.append(base_url + href)
        return file_urls
    return []

# CSV 파일들이 있는 기본 URL
base_url = "https://people.sc.fsu.edu/~jburkardt/data/csv/"
# 모든 CSV 파일 URL을 가져옵니다.
file_urls = get_all_csv_urls(base_url)

### **동기와 비동기의 차이**

In [ ]:
# 동기 방식으로 파일을 다운로드하는 함수
def download_file_sync(url, save_path):
    # 주어진 URL로부터 파일을 요청합니다.
    response = requests.get(url)
    if response.status_code == 200:
        # 파일 내용을 지정된 경로에 저장합니다.
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print(f"{save_path} 다운로드 완료")

# 모든 파일을 동기 방식으로 다운로드하는 함수
def download_all_files_sync():
    # file_urls에 있는 각 파일을 순차적으로 다운로드합니다.
    for url in file_urls:
        filename = os.path.basename(url)
        save_path = f"./{filename}"
        download_file_sync(url, save_path)

# 비동기 방식으로 파일을 다운로드하는 함수
async def download_file_async(session, url, save_path):
    # 비동기적으로 주어진 URL로부터 파일을 요청합니다.
    async with session.get(url) as response:
        if response.status == 200:
            # 파일 내용을 지정된 경로에 저장합니다.
            content = await response.read()
            with open(save_path, 'wb') as f:
                f.write(content)
            print(f"{save_path} 다운로드 완료")

# 모든 파일을 비동기 방식으로 다운로드하는 함수
async def download_all_files():
    # aiohttp.ClientSession을 사용하여 비동기 요청을 관리합니다.
    async with aiohttp.ClientSession() as session:
        tasks = []
        # file_urls에 있는 각 파일을 비동기적으로 다운로드하도록 태스크를 만듭니다.
        for url in file_urls:
            filename = os.path.basename(url)
            save_path = f"./{filename}"
            tasks.append(download_file_async(session, url, save_path))
        # 모든 태스크를 동시에 실행합니다.
        await asyncio.gather(*tasks)

# 동기 방식으로 파일 다운로드 시작
print("동기 방식 순서대로 다운로드 시작...")
t1 = time.time()
download_all_files_sync()
t2 = time.time()
print(f"동기 방식으로 파일 다운로드 완료: {t2 - t1:.2f}초 소요")

# 비동기 방식으로 파일 다운로드 시작
# asyncio.run() 대신 IPython event loop에서 실행하기 위해 await 사용
t1 = time.time()
# await download_all_files()  # Jupyter/IPython 환경에서 실행하기 위해 await 사용
# asyncio.run(download_all_files())  # 이 줄을 주석 처리
await download_all_files() # asyncio.run()을 await로 변경하여 기존 event loop에서 실행
t2 = time.time()
print(f"비동기 방식으로 파일 다운로드 완료: {t2 - t1:.2f}초 소요")

동기 방식 순서대로 다운로드 시작...
./addresses.csv 다운로드 완료
./airtravel.csv 다운로드 완료
./biostats.csv 다운로드 완료
./cities.csv 다운로드 완료
./crash_catalonia.csv 다운로드 완료
./deniro.csv 다운로드 완료
./example.csv 다운로드 완료
./faithful.csv 다운로드 완료
./ford_escort.csv 다운로드 완료
./freshman_kgs.csv 다운로드 완료
./freshman_lbs.csv 다운로드 완료
./grades.csv 다운로드 완료
./homes.csv 다운로드 완료
./hooke.csv 다운로드 완료
./hurricanes.csv 다운로드 완료
./hw_200.csv 다운로드 완료
./hw_25000.csv 다운로드 완료
./lead_shot.csv 다운로드 완료
./letter_frequency.csv 다운로드 완료
./mlb_players.csv 다운로드 완료
./mlb_teams_2012.csv 다운로드 완료
./news_decline.csv 다운로드 완료
./nile.csv 다운로드 완료
./oscar_age_female.csv 다운로드 완료
./oscar_age_male.csv 다운로드 완료
./snakes_count_10.csv 다운로드 완료
./snakes_count_100.csv 다운로드 완료
./snakes_count_1000.csv 다운로드 완료
./snakes_count_10000.csv 다운로드 완료
./tally_cab.csv 다운로드 완료
./taxables.csv 다운로드 완료
./trees.csv 다운로드 완료
./zillow.csv 다운로드 완료
동기 방식으로 파일 다운로드 완료: 21.33초 소요
./nile.csv 다운로드 완료
./homes.csv 다운로드 완료
./zillow.csv 다운로드 완료
./mlb_teams_2012.csv 다운로드 완료
./faithful.csv 다운로드 완료
./letter

## 여러 파일을 동시에 다운로드하므로 네트워크 요청의 병렬 처리가 가능
- 특히 네트워크 대역폭을 충분히 활용하여 다운로드 시간을 줄임
- aiohttp와 asyncio를 사용하여 비동기적으로 네트워크 요청을 처리
- Jupyter 환경에서는 await로 직접 실행할 수 있으며, 일반 파이썬 스크립트에서는 asyncio.run()을 사용해 실행 가능

In [ ]:
import asyncio

async def say_hello():
    print("Hello")
    await asyncio.sleep(5)  # 5초 동안 기다리며 다른 작업을 수행할 수 있음
    #  "1초 동안 기다리며 다른 작업을 수행할 수 있다"는 것은, 1초 동안 기다리는 동안 CPU가 유휴 상태로 멈추는 대신, 다른 비동기 작업에 집중할 수 있다는 뜻
    print("World")

# Jupyter Notebook에서 실행 중인 이벤트 루프에서 직접 await로 호출
await say_hello()

Hello
World


## **4. 비동기 모듈에 대하여**

- **aiohttp:** 비동기적으로 HTTP 요청을 보낼 수 있도록 도와주는 라이브러리
- **asyncio:** 비동기적으로 I/O 작업을 관리하기 위한 Python 표준 라이브러리로, 이벤트 루프를 통해 여러 작업을 효율적으로 스케줄링하고 처리

In [ ]:
import aiohttp  # 비동기 HTTP 클라이언트 라이브러리
import asyncio  # 비동기 I/O를 위한 Python 표준 라이브러리

- async def fetch(url): fetch 함수는 비동기 함수로 정의되어 있으며, 특정 URL에서 HTTP GET 요청
- async with aiohttp.ClientSession() as session: 비동기적으로 HTTP 세션을 생성하며 ClientSession 객체를 사용하면 여러 요청 간 세션을 재사용할 수 있어 효율적
- async with session.get(url) as response: 지정된 URL로 비동기적으로 HTTP GET 요청을 보내고, 응답을 기다린다
- await response.text(): 응답의 내용을 텍스트 형태로 비동기적으로 읽고 반환, await를 사용하여 읽기가 완료될 때까지 대기합니다.

In [ ]:
# 특정 URL에서 HTML 데이터를 가져오는 비동기 함수 정의
async def fetch(url):
    # 비동기적으로 HTTP 세션을 시작 (세션은 여러 요청 간 재사용할 수 있는 컨텍스트 관리용 객체)

    async with aiohttp.ClientSession() as session:
        # 지정된 URL로 GET 요청을 비동기적으로 보내고 응답을 받음

        async with session.get(url) as response:
            # 응답의 본문을 비동기적으로 읽고 반환 (await를 사용해 응답 본문 대기)
            # 웹 페이지에서 데이터를 다운로드하고자 할 때 **await**은 "서버로부터 데이터가 도착할 때까지 잠시 기다리세요"라고 Python에게 말하는 것
            # 이 동안, Python은 "서버에서 데이터가 오는 동안 CPU가 아무것도 하지 않으며 멍하니 기다리는 것" 대신 "다른 작업도 처리할 수 있도록 기회를 준다"는 것
            # '기다려'의 비유 : 친구(서버)에게 커피를 가져다 달라고 부탁하고 커피가 오는 동안 나는 다른 일(전화, 서서 운동 등등)을 본다.
            return await response.text()


- async def main(): main 함수는 비동기 함수로 정의되어 있으며, 실제로 fetch() 함수를 호출하여 응답을 받아옵니다.
- url = 'https://example.com': 요청을 보낼 대상 URL을 설정합니다.
- html = await fetch(url): fetch() 함수를 호출하고, 응답 본문을 html 변수에 저장합니다.
- print(html): 응답 받은 HTML을 콘솔에 출력합니다.

In [ ]:
# main() 함수 정의 - 실제로 fetch 함수를 호출하고 결과를 출력
async def main():
    url = 'https://example.com'  # 요청을 보낼 URL 설정
    html = await fetch(url)  # fetch() 함수를 호출해 응답 본문을 받음
    print(html)  # 응답 받은 HTML 내용 출력

In [ ]:
# Jupyter 환경에서는 이미 이벤트 루프가 실행 중이므로 await을 직접 사용해 main()을 호출
await main()  # main() 함수를 호출해 비동기 작업 실행 및 결과 출력

<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        div {
            margin: 0 auto;
            width: auto;
        }
    }
    </style>    
</head>

<body>
<div>
    <h1>Example Domain</h1>
    <p>This domai

### **aiohttp는 Python의 HTTP 클라이언트 및 서버를 비동기적으로 구현할 수 있는 HTTP 클라이언트 라이브러리**
- asyncio와 결합하여 비동기적으로 HTTP 요청을 보낼 수 있도록 지원
- 일반적인 HTTP 요청 라이브러리인 requests와 달리, aiohttp는 비동기 처리가 가능하므로 수백 개 이상의 요청을 동시에 처리할 수 있는 경우에 매우 유리